 # 1) Import Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# 2) Device

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 3) Transform

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))
])

# 4) Dataset

In [4]:
trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True,
    download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False,
    download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=64, shuffle=True)

testloader = torch.utils.data.DataLoader(
    testset, batch_size=64, shuffle=False)

100%|██████████| 170M/170M [00:01<00:00, 88.9MB/s]


# 5) CNN Model

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(64 * 6 * 6, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))  # 32→30→15
        x = self.pool(torch.relu(self.conv2(x)))  # 15→13→6
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CNN().to(device)


# 6) Loss & Optimizer

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 7) Training

In [7]:
epochs = 5
for epoch in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss:.4f}")

print("Training Finished")

Epoch [1/5], Loss: 1092.6513
Epoch [2/5], Loss: 809.6092
Epoch [3/5], Loss: 688.1396
Epoch [4/5], Loss: 597.3832
Epoch [5/5], Loss: 529.4350
Training Finished
